In [3]:
import pandas as pd

trade_df_path = '../data/kr_trade_data_by_country.csv'
# trade_df_path = '../data/kr_trade_data_total.csv'

trade_df = pd.read_csv(trade_df_path, encoding='utf-8')
trade_df.tail(20)

,Date,export_amount,import_amount,trade_balance,country_name
89048,2025-05-31,768863,0,768863,과델로프
89049,2025-05-31,777606,18368,759238,적도 기니
89050,2025-05-31,792217,185247,606970,불령 폴리네시아
89051,2025-05-31,793937,597672,196265,부탄
89052,2025-05-31,810186,73578,736608,마이크로네시아
89053,2025-05-31,1306467,10405641,-9099174,잠비아
89054,2025-05-31,825328,288221,537107,세이쉘
89055,2025-05-31,866109,80301,785808,북마리아나 군도
89056,2025-05-31,866821,556041,310780,마세도니아
89057,2025-05-31,878607,1667,876940,투발루


In [ ]:
def get_ttm_yoy_only_df(trade_df):
    # 1. Date 컬럼을 datetime으로 변환하고 인덱스로 설정 (필요시)
    if not pd.api.types.is_datetime64_any_dtype(trade_df['Date']):
        trade_df['Date'] = pd.to_datetime(trade_df['Date'])
    trade_df = trade_df.set_index('Date')

    # 2. TTM(12개월 rolling sum) 계산
    ttm_df = (
        trade_df
        .groupby('country_name')[['export_amount', 'import_amount', 'trade_balance']]
        .rolling(window=12, min_periods=12).sum()
        .reset_index()
    )
    ttm_df.rename(columns={
        'export_amount': 'ttm_export_amount',
        'import_amount': 'ttm_import_amount',
        'trade_balance': 'ttm_trade_balance'
    }, inplace=True)
    ttm_df = ttm_df.dropna()

    # 3. TTM의 YoY(%) 계산 (접두사 ttm_yoy_)
    ttm_yoy_df = ttm_df[['country_name', 'Date']].copy()
    for col in ['ttm_export_amount', 'ttm_import_amount', 'ttm_trade_balance']:
        ttm_yoy_df[f'ttm_yoy_{col[4:]}'] = (
            ttm_df.groupby('country_name')[col]
            .pct_change(periods=12) * 100
        )
    # 4. 12개월 전 데이터가 없는 row 제거 (NaN drop)
    ttm_yoy_df = ttm_yoy_df.dropna()

    return ttm_yoy_df

# 사용 예시
ttm_yoy_only_df = get_ttm_yoy_only_df(trade_df)
ttm_yoy_only_df.tail()

저장 완료:
../data\ttm_kr_trade_data_by_country.csv
../data\yoy_kr_trade_data_by_country.csv
../data\ttm_yoy_kr_trade_data_by_country.csv


In [5]:
import pandas as pd
import os

def save_ttm_yoy_ttm_yoy_df(trade_df, filename):
    # 1. Date 컬럼을 datetime으로 변환하고 인덱스로 설정 (필요시)
    if not pd.api.types.is_datetime64_any_dtype(trade_df['Date']):
        trade_df['Date'] = pd.to_datetime(trade_df['Date'])
    trade_df = trade_df.set_index('Date')

    # 2. TTM(12개월 rolling sum) 계산
    ttm_df = (
        trade_df
        .groupby('country_name')[['export_amount', 'import_amount', 'trade_balance']]
        .rolling(window=12, min_periods=12).sum()
        .reset_index()
    )
    ttm_df.rename(columns={
        'export_amount': 'ttm_export_amount',
        'import_amount': 'ttm_import_amount',
        'trade_balance': 'ttm_trade_balance'
    }, inplace=True)
    ttm_df = ttm_df.dropna()

    # 3. YoY(%) 계산 (원본값 기준)
    yoy_df = ttm_df[['country_name', 'Date']].copy()
    for col in ['export_amount', 'import_amount', 'trade_balance']:
        # 원본값 가져오기
        yoy_df[f'yoy_{col}'] = (
            trade_df.reset_index().set_index(['country_name', 'Date'])[col].reindex(
                ttm_df.set_index(['country_name', 'Date']).index
            ).values
        )
        # YoY 계산
        yoy_df[f'yoy_{col}'] = (
            yoy_df.groupby('country_name')[f'yoy_{col}']
            .pct_change(periods=12) * 100
        )
    yoy_df = yoy_df.dropna()

    # 4. TTM의 YoY(%) 계산
    ttm_yoy_df = ttm_df[['country_name', 'Date']].copy()
    for col in ['ttm_export_amount', 'ttm_import_amount', 'ttm_trade_balance']:
        ttm_yoy_df[f'ttm_yoy_{col[4:]}'] = (
            ttm_df.groupby('country_name')[col]
            .pct_change(periods=12) * 100
        )
    ttm_yoy_df = ttm_yoy_df.dropna()

    # 5. 파일명 앞에 접두사 붙이기
    base_dir = os.path.dirname(filename)
    base_name = os.path.basename(filename)
    ttm_path = os.path.join(base_dir, 'ttm_' + base_name)
    yoy_path = os.path.join(base_dir, 'yoy_' + base_name)
    ttm_yoy_path = os.path.join(base_dir, 'ttm_yoy_' + base_name)

    # 6. 저장
    ttm_df.round(1).to_csv(ttm_path, index=False, encoding='utf-8-sig')
    yoy_df.round(1).to_csv(yoy_path, index=False, encoding='utf-8-sig')
    ttm_yoy_df.round(1).to_csv(ttm_yoy_path, index=False, encoding='utf-8-sig')
    print(f"저장 완료:\n{ttm_path}\n{yoy_path}\n{ttm_yoy_path}")

# 사용 예시
save_ttm_yoy_ttm_yoy_df(trade_df, trade_df_path)

저장 완료:
../data\ttm_kr_trade_data_by_country.csv
../data\yoy_kr_trade_data_by_country.csv
../data\ttm_yoy_kr_trade_data_by_country.csv
